In [1]:
# Imports required libraries
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import hvplot
from pathlib import Path

%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
# Sets keys
alpaca_api = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_api)
type(alpaca_secret)

# Creates the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api,
    alpaca_secret,
    api_version="v2"
)

In [4]:
# Set timeframe to 1Day
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2020-04-21", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-04-21", tz="America/New_York").isoformat()


In [5]:
# Creates the list of tickers that the user inputs

tickers = []

tickers.append(input("Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY)) "))
tickers.append(input("Enter your second ticker."))
tickers.append(input("Enter your third ticker."))
tickers.append(input("Enter your fourth ticker."))
tickers.append(input("Enter your fifth ticker."))
print(f"The tickers you have selected are: {tickers}")


Enter your first stock or bond ticker (as it appears on the exchange (example: BBBY))  SPY
Enter your second ticker. AMZN
Enter your third ticker. MSFT
Enter your fourth ticker. GOOG
Enter your fifth ticker. GME


The tickers you have selected are: ['SPY', 'AMZN', 'MSFT', 'GOOG', 'GME']


In [6]:
# Uses the Alpaca get_bars function to get current closing prices of the portfolio and creates
# a dataframe
stock_bond = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
    
# Reorganizes the DataFrame
# Separates ticker data
ticker_1 = stock_bond[stock_bond['symbol']==stock_bond.iloc[:,7]].drop('symbol', axis=1)
ticker_2 = stock_bond[stock_bond['symbol']==stock_bond.iloc[:,7]].drop('symbol', axis=1)
ticker_3 = stock_bond[stock_bond['symbol']==stock_bond.iloc[:,7]].drop('symbol', axis=1)
ticker_4 = stock_bond[stock_bond['symbol']==stock_bond.iloc[:,7]].drop('symbol', axis=1)
ticker_5 = stock_bond[stock_bond['symbol']==stock_bond.iloc[:,7]].drop('symbol', axis=1)

# Concatenates the ticker DataFrames
stock_bond_df = pd.concat([ticker_1, ticker_2, ticker_3, ticker_4, ticker_5], axis=1, keys=[tickers[0], tickers[1], tickers[2], tickers[3], tickers[4]])

# Reviews the Alpaca DataFrame
stock_bond_df.head()

SPY                                            \
                              open       high        low    close     volume   
timestamp                                                                      
2020-04-21 04:00:00+00:00  2417.50  2428.3099  2279.6600  2328.12    7476696   
2020-04-21 04:00:00+00:00     5.23     5.3000     4.7600     4.78    4142054   
2020-04-21 04:00:00+00:00  1246.27  1254.2700  1209.7100  1216.34    2153003   
2020-04-21 04:00:00+00:00   173.50   173.6699   166.1102   167.82   56203764   
2020-04-21 04:00:00+00:00   276.48   278.0400   272.0200   273.04  126387834   

                                                       AMZN             \
                          trade_count         vwap     open       high   
timestamp                                                                
2020-04-21 04:00:00+00:00      272681  2343.749939  2417.50  2428.3099   
2020-04-21 04:00:00+00:00       20092     4.977210     5.23     5.3000   
2020-04-21 04:00:00+00:00       81297  1225.058231  1246.27  1254.2700   
2020-04-21 04:00:00+00:00      531392   169.058519   173.50   173.6699   
2020-04-21 04:00:00+00:00      923555   274.755324   276.48   278.0400   

                                      ...       GOOG                           \
                                 low  ...     volume trade_count         vwap   
timestamp                             ...                                       
2020-04-21 04:00:00+00:00  2279.6600  ...    7476696      272681  2343.749939   
2020-04-21 04:00:00+00:00     4.7600  ...    4142054       20092     4.977210   
2020-04-21 04:00:00+00:00  1209.7100  ...    2153003       81297  1225.058231   
2020-04-21 04:00:00+00:00   166.1102  ...   56203764      531392   169.058519   
2020-04-21 04:00:00+00:00   272.0200  ...  126387834      923555   274.755324   

                               GME                                            \
                              open       high        low    close     volume   
timestamp                                                                      
2020-04-21 04:00:00+00:00  2417.50  2428.3099  2279.6600  2328.12    7476696   
2020-04-21 04:00:00+00:00     5.23     5.3000     4.7600     4.78    4142054   
2020-04-21 04:00:00+00:00  1246.27  1254.2700  1209.7100  1216.34    2153003   
2020-04-21 04:00:00+00:00   173.50   173.6699   166.1102   167.82   56203764   
2020-04-21 04:00:00+00:00   276.48   278.0400   272.0200   273.04  126387834   

                                                    
                          trade_count         vwap  
timestamp                                           
2020-04-21 04:00:00+00:00      272681  2343.749939  
2020-04-21 04:00:00+00:00       20092     4.977210  
2020-04-21 04:00:00+00:00       81297  1225.058231  
2020-04-21 04:00:00+00:00      531392   169.058519  
2020-04-21 04:00:00+00:00      923555   274.755324  

[5 rows x 35 columns]

In [7]:
# Reads the S&P 500 data into a dataframe
sp500 = pd.read_csv(
    Path("Resources/Download Data - INDEX_US_S&P US_SPX.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True
)

In [8]:
# Displays the data
display(sp500.head())
display(sp500.tail())

,Open,High,Low,Close
Date,,,,
2022-05-10,"4,035.18","4,068.82","3,958.17","4,001.05"
2022-05-09,"4,081.27","4,081.27","3,975.48","3,991.24"
2022-05-06,"4,128.17","4,157.69","4,067.91","4,123.34"
2022-05-05,"4,270.43","4,270.43","4,106.01","4,146.87"
2022-05-04,"4,181.18","4,307.66","4,148.91","4,300.17"


,Open,High,Low,Close
Date,,,,
2021-05-14,"4,129.58","4,183.13","4,129.58","4,173.85"
2021-05-13,"4,074.99","4,131.58","4,074.99","4,112.50"
2021-05-12,"4,130.55","4,134.73","4,056.88","4,063.04"
2021-05-11,"4,150.34","4,162.04","4,111.53","4,152.10"
2021-05-10,"4,228.29","4,236.39","4,188.13","4,188.43"


In [9]:
# Checks for null values in the data
sp500.isnull().sum()

Open     0
High     0
Low      0
Close    0
dtype: int64

In [10]:
# Cleans and prepares the data for the dataframe
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].str.replace(",", "")
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].str.replace(",", "")
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].str.replace(",", "")
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].str.replace(",", "")

# Confirms cleaning of data
sp500.head()

,Open,High,Low,Close
Date,,,,
2022-05-10,4035.18,4068.82,3958.17,4001.05
2022-05-09,4081.27,4081.27,3975.48,3991.24
2022-05-06,4128.17,4157.69,4067.91,4123.34
2022-05-05,4270.43,4270.43,4106.01,4146.87
2022-05-04,4181.18,4307.66,4148.91,4300.17


In [11]:
# Changes all data types to float
sp500.loc[:, 'Open'] = sp500.loc[:, 'Open'].astype('float')
sp500.loc[:, 'High'] = sp500.loc[:, 'High'].astype('float')
sp500.loc[:, 'Low'] = sp500.loc[:, 'Low'].astype('float')
sp500.loc[:, 'Close'] = sp500.loc[:, 'Close'].astype('float')

# Confirms data type conversion
sp500.dtypes

Open     float64
High     float64
Low      float64
Close    float64
dtype: object